# Pynamical: animated 3D phase diagrams of the logistic map

Author: Geoff Boeing

http://geoffboeing.com/2015/04/animated-3d-plots-python/

This notebook demonstrates how to make animated GIFs that pan and zoom around 3-D phase diagrams to visualize fractal data sets, strange attractors, and chaos. If you get errors from images2gif, check out [this page](https://stackoverflow.com/questions/19149643/error-in-images2gif-py-with-globalpalette) or replace images2.gif.py with [this version](https://github.com/rec/echomesh/blob/master/code/python/external/images2gif.py). If you're on Python 3 you may need [this version](https://github.com/isaacgerg/images2gif) of images2gif.

In [1]:
from pynamical import simulate, phase_diagram_3d, title_font, label_font
import pandas as pd, numpy as np, matplotlib.pyplot as plt, random, glob, IPython.display as IPdisplay
from PIL import Image as PIL_Image
from images2gif import writeGif
%matplotlib inline

In [2]:
save_folder = 'images/animation/'

## Create a 3-D phase diagram as an animated gif that pans, rotates, and zooms. This demonstrates how the viewing perspective is composed of an elevation, a distance, and an azimuth.

In [3]:
# set a filename, run the logistic model, and create the plot
gif_filename = 'pan-rotate-zoom-demo'
pops = simulate(num_gens=1000, rate_min=3.99, num_rates=1)
fig, ax = phase_diagram_3d(pops, remove_ticks=False, show=False, save=False)

# create 36 frames for the animated gif
steps = 36

# a viewing perspective is composed of an elevation, distance, and azimuth
# define the range of values we'll cycle through for the distance of the viewing perspective
min_dist = 7.
max_dist = 10.
dist_range = np.arange(min_dist, max_dist, (max_dist-min_dist)/steps)

# define the range of values we'll cycle through for the elevation of the viewing perspective
min_elev = 10.
max_elev = 60.
elev_range = np.arange(max_elev, min_elev, (min_elev-max_elev)/steps)

# now create the individual frames that will be combined later into the animation
for azimuth in range(0, 360, int(360/steps)):
    
    # pan down, rotate around, and zoom out
    ax.azim = float(azimuth/3.)
    ax.elev = elev_range[int(azimuth/(360./steps))]
    ax.dist = dist_range[int(azimuth/(360./steps))]
    
    # set the figure title to the viewing perspective, and save each figure as a .png
    fig.suptitle('elev=' + str(round(ax.elev,1)) + ', azim=' + str(round(ax.azim,1)) + ', dist=' + str(round(ax.dist,1)))
    plt.savefig(save_folder + gif_filename + '/img' + str(azimuth).zfill(3) + '.png')
    
# don't display the static plot...
plt.close()

# ...instead, create an animated gif of all the frames, then display it inline
images = [PIL_Image.open(image) for image in glob.glob(save_folder + gif_filename + '/*.png')]
file_path_name = save_folder + gif_filename + '.gif'
writeGif(file_path_name, images, duration=0.2)
IPdisplay.Image(url=file_path_name)

## Create a 3-D phase diagram as an animated gif starts by looking straight down at the x-y plane (this is what a 2-D plot would look like), then panning and rotating around to show the 3-D structure

In [4]:
# set a filename, run the logistic model, and create the plot
gif_filename = 'pan-rotate-logistic-phase-diagram'
pops = simulate(num_gens=1000, rate_min=3.99, num_rates=1)
fig, ax = phase_diagram_3d(pops, color='#003399', xlabel='Population (t)', ylabel='Population (t + 1)', zlabel='', 
                           show=False, save=False)

# look straight down at the x-y plane to start off
ax.elev = 89.9
ax.azim = 270.1
ax.dist = 11.0

# sweep the perspective down and rotate to reveal the 3-D structure of the strange attractor
for n in range(0, 100):
    if n > 19 and n < 23:
        ax.set_xlabel('')
        ax.set_ylabel('') #don't show axis labels while we move around, it looks weird
        ax.elev = ax.elev-0.5 #start by panning down slowly
    if n > 22 and n < 37:
        ax.elev = ax.elev-1.0 #pan down faster
    if n > 36 and n < 61:
        ax.elev = ax.elev-1.5
        ax.azim = ax.azim+1.1 #pan down faster and start to rotate
    if n > 60 and n < 65:
        ax.elev = ax.elev-1.0
        ax.azim = ax.azim+1.1 #pan down slower and rotate same speed
    if n > 64 and n < 74:
        ax.elev = ax.elev-0.5
        ax.azim = ax.azim+1.1 #pan down slowly and rotate same speed
    if n > 73 and n < 77:
        ax.elev = ax.elev-0.2
        ax.azim = ax.azim+0.5 #end by panning/rotating slowly to stopping position   
    if n > 76: #add axis labels at the end, when the plot isn't moving around
        ax.set_xlabel('Population (t)')
        ax.set_ylabel('Population (t + 1)')
        ax.set_zlabel('Population (t + 2)')
        
    # add a figure title to each plot then save the figure to the disk
    fig.suptitle('Logistic Map, r=3.99', fontsize=16, x=0.5, y=0.85)
    plt.savefig(save_folder + gif_filename + '/img' + str(n).zfill(3) + '.png', bbox_inches='tight')

# don't display the static plot
plt.close()

# create an animated gif of all the 3-D plot perspectives then display it inline
images = [PIL_Image.open(image) for image in glob.glob(save_folder + gif_filename + '/*.png')]
file_path_name = save_folder + gif_filename + '.gif'
writeGif(file_path_name, images, duration=0.1)
IPdisplay.Image(url=file_path_name)

## Do the same thing again, but this time plot both the chaotic logistic model output and random noise

In [5]:
# run the logistic model and create random noise
chaos_pops = simulate(num_gens=1000, rate_min=3.99, num_rates=1)
random_pops = pd.DataFrame([random.random() for _ in range(0, 1000)], columns=['value'])
pops = pd.concat([chaos_pops, random_pops], axis=1)
pops.columns = ['chaos', 'random']
pops.tail()

,chaos,random
995,0.247214,0.324087
996,0.742535,0.091102
997,0.762795,0.001077
998,0.721947,0.206058
999,0.800951,0.786185


In [6]:
# set a filename and then create the plot
gif_filename = 'pan-rotate-logistic-random'
fig, ax = phase_diagram_3d(pops, color=['#003399','#cc0000'], xlabel='Population (t)', ylabel='Population (t + 1)', 
                           zlabel='', legend=True, legend_bbox_to_anchor=(0.94, 0.96), show=False, save=False)

# configure the initial viewing perspective to look straight down at the x-y plane
ax.elev = 89.9
ax.azim = 270.1
ax.dist = 11.0

# sweep the perspective down and rotate to reveal the 3-D structure of the strange attractor
for n in range(0, 100):
    if n >= 20 and n <= 22:
        ax.set_xlabel('')
        ax.set_ylabel('') #don't show axis labels while we move around, it looks weird
        ax.elev = ax.elev-0.5 #start by panning down slowly
    if n >= 23 and n <= 36:
        ax.elev = ax.elev-1.0 #pan down faster
    if n >= 37 and n <= 60:
        ax.elev = ax.elev-1.5
        ax.azim = ax.azim+1.1 #pan down faster and start to rotate
    if n >= 61 and n <= 64:
        ax.elev = ax.elev-1.0
        ax.azim = ax.azim+1.1 #pan down slower and rotate same speed
    if n >= 65 and n <= 73:
        ax.elev = ax.elev-0.5
        ax.azim = ax.azim+1.1 #pan down slowly and rotate same speed
    if n >= 74 and n <= 76:
        ax.elev = ax.elev-0.2
        ax.azim = ax.azim+0.5 #end by panning/rotating slowly to stopping position   
    if n == 77: #add axis labels at the end, when the plot isn't moving around
        ax.set_xlabel('Population (t)')
        ax.set_ylabel('Population (t + 1)')
        ax.set_zlabel('Population (t + 2)')
        
    # add a figure title to each plot then save the figure to the disk
    fig.suptitle(u'3-D phase diagram, chaos vs random', fontsize=16, x=0.5, y=0.85)
    plt.savefig(save_folder + gif_filename + '/img' + str(n).zfill(3) + '.png', bbox_inches='tight')

# don't display the static plot
plt.close()

# create an animated gif of all the 3-D plot perspectives then display it inline
images = [PIL_Image.open(image) for image in glob.glob(save_folder + gif_filename + '/*.png')]
file_path_name = save_folder + gif_filename + '.gif'
writeGif(file_path_name, images, duration=0.1)
IPdisplay.Image(url=file_path_name)

## Create a 3-D phase diagram to show the logistic map's strange attractors across the chaotic regime (from r=3.6 to r=4.0), twisting and curling around their state space in three dimensions. Animated it by panning and rotating to reveal the structure and its odd folds.

In [7]:
# run the model for 2,000 generations for 50 growth rate parameters between 3.6 and 4.0
pops = simulate(num_gens=2000, rate_min=3.6, rate_max=4.0, num_rates=50)

In [8]:
# set a filename and create the plot
gif_filename = 'pan-rotate-chaotic-regime'
fig, ax = phase_diagram_3d(pops, color='viridis', color_reverse=False,
                           xlabel='Population (t)', ylabel='Population (t + 1)', zlabel='', 
                           show=False, save=False)

# configure the initial viewing perspective to look straight down at the x-y plane
ax.elev = 89.9
ax.azim = 270.1
ax.dist = 11.0

# sweep the perspective down and rotate to reveal the 3-D structure of the strange attractor
for n in range(0, 100):
    if n > 19 and n < 23:
        ax.set_xlabel('')
        ax.set_ylabel('') #don't show axis labels while we move around, it looks weird
        ax.elev = ax.elev-0.5 #start by panning down slowly
    if n > 22 and n < 37:
        ax.elev = ax.elev-1.0 #pan down faster
    if n > 36 and n < 61:
        ax.elev = ax.elev-1.5
        ax.azim = ax.azim+1.1 #pan down faster and start to rotate
    if n > 60 and n < 65:
        ax.elev = ax.elev-1.0
        ax.azim = ax.azim+1.1 #pan down slower and rotate same speed
    if n > 64 and n < 74:
        ax.elev = ax.elev-0.5
        ax.azim = ax.azim+1.1 #pan down slowly and rotate same speed
    if n > 73 and n < 77:
        ax.elev = ax.elev-0.2
        ax.azim = ax.azim+0.5 #end by panning/rotating slowly to stopping position
    
    if n > 76: #add axis labels at the end, when the plot isn't moving around
        ax.set_xlabel('Population (t)')
        ax.set_ylabel('Population (t + 1)')
        ax.set_zlabel('Population (t + 2)')
    
    # add a figure title to each plot then save the figure to the disk
    fig.suptitle('Logistic Map, r=3.6 to r=4.0', fontsize=16, x=0.5, y=0.85)
    plt.savefig(save_folder + gif_filename + '/img' + str(n).zfill(3) + '.png', bbox_inches='tight')

# don't display the static plot
plt.close()

# create an animated gif of all the 3-D plot perspectives then display it inline
images = [PIL_Image.open(image) for image in glob.glob(save_folder + gif_filename + '/*.png')]
file_path_name = save_folder + gif_filename + '.gif'
writeGif(file_path_name, images, duration=0.1)
IPdisplay.Image(url=file_path_name)

## Now zoom into the 3D plot

In [9]:
# run the model for 4,000 generations for 50 growth rate parameters between 3.6 and 4.0
pops = simulate(num_gens=4000, rate_min=3.6, rate_max=4.0, num_rates=50)

In [10]:
# set a filename and create the plot
gif_filename = 'logistic-3d-phase-diagram-chaotic-regime'
fig, ax = phase_diagram_3d(pops, color='viridis', color_reverse=False, show=False, save=False)

# configure the initial viewing perspective
ax.elev = 25.
ax.azim = 321.
ax.dist = 11.0

# zoom in to reveal the 3-D structure of the strange attractor
for n in range(0, 100):
    if n <= 18:
        ax.azim = ax.azim-0.2 #begin by rotating very slowly
    if n >= 19 and n <= 29:
        ax.azim = ax.azim-10
        ax.dist = ax.dist-0.05
        ax.elev = ax.elev-2 #quickly whip around to the other side
    if n >= 33 and n <= 49:
        ax.azim = ax.azim+3
        ax.dist = ax.dist-0.55
        ax.elev = ax.elev+1.4 #zoom into the center
    if n >= 61 and n <= 79:
        ax.azim = ax.azim-2
        ax.elev = ax.elev-2
        ax.dist = ax.dist+0.2 #pull back and pan up
    if n >= 80:
        ax.azim = ax.azim-0.2 #end by rotating very slowly
    
    # add a figure title to each plot then save the figure to the disk
    fig.suptitle('Logistic Map, r=3.6 to r=4.0', fontsize=16, x=0.5, y=0.85)
    plt.savefig(save_folder + gif_filename + '/img' + str(n).zfill(3) + '.png', bbox_inches='tight')

# don't display the static plot
plt.close()

# create an animated gif of all the 3-D plot perspectives then display it inline
images = [PIL_Image.open(image) for image in glob.glob(save_folder + gif_filename + '/*.png')]
file_path_name = save_folder + gif_filename + '.gif'
writeGif(file_path_name, images, duration=0.1)
IPdisplay.Image(url=file_path_name)

## For more information about chaos theory, the logistic map, bifurcation plots, phase diagrams, and strange attractors, check out my write-up:
http://geoffboeing.com/2015/03/chaos-theory-logistic-map/
<br />and<br />
http://geoffboeing.com/2015/04/visualizing-chaos-and-randomness/
<br />and<br />
http://geoffboeing.com/2015/04/animated-3d-plots-python/